In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-05-09 03:55:51.418756: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-09 03:55:52.920622: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-09 03:55:54.198492: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-09 03:55:54.198945

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

def extract_events(article):
    # Parsing the article with spaCy
    doc = nlp(article)

    # Initializing lists for storing extracted events and entities
    events = []
    entities = []

    # Iterate over the sentences in the article
    for sent in doc.sents:
        # Extracting the subject, verb, and object of each sentence
        subj=''
        verb=''
        obj=''
        for token in sent:
            if token.dep_ == "nsubj":
                subj = token.text
                subj_start = token.idx
                subj_end = token.idx + len(token)
            elif token.dep_ == "ROOT" and token.pos_ == "VERB":
                verb = token.text
                verb_start = token.idx
                verb_end = token.idx + len(token)
            elif token.dep_ == "dobj":
                obj = token.text
                obj_start = token.idx
                obj_end = token.idx + len(token)

        # If we have a complete event, add it to the list
        if all(x is not None for x in [subj, verb, obj]):
            events.append({"event": f"{subj} {verb} {obj}"})
                          #  "start": min(subj_start, verb_start, obj_start),
                          #  "end": max(subj_end, verb_end, obj_end)})
            
        # Extracting named entities from the sentence
        for ent in sent.ents:
            entities.append({"entity": ent.text,
                            #  "start": ent.start_char,
                            #  "end": ent.end_char,
                             "label": ent.label_})
    
    return events, entities


In [ ]:
import pandas as pd

In [ ]:
df =pd.read_csv('/content/crime_articles.csv')

In [ ]:
df.head()

,heading,content_summary,article_link,img_link,month_date,time,Year
0,"Uttarakhand: Man, 39, held for killing consta...",Nainital district police on Monday arrested a...,https://timesofindia.indiatimes.com/city/dehra...,https://static.toiimg.com/thumb/imgsize-123456...,Nov 08,09:55,2022
1,"Nashik city: Another theft from parked car, s...",Thefts from parked cars continues in the city...,https://timesofindia.indiatimes.com/city/nashi...,https://static.toiimg.com/thumb/imgsize-123456...,Nov 08,08:25,2022
2,Pune: Externed criminal jumps from bus fearin...,An externed criminal died in the early hours ...,https://timesofindia.indiatimes.com/city/pune/...,https://static.toiimg.com/thumb/imgsize-123456...,Nov 08,05:17,2022
3,Gaya woman burned alive on suspicion of witch...,A middle-aged woman was badly thrashed and bu...,https://timesofindia.indiatimes.com/city/patna...,https://static.toiimg.com/thumb/imgsize-123456...,Nov 06,05:24,2022
4,Man held on charge of stalking in Pune,A man (33) was arrested on Tuesday from Khed ...,https://timesofindia.indiatimes.com/city/pune/...,https://static.toiimg.com/thumb/imgsize-123456...,Nov 03,08:32,2022


In [ ]:
df.iloc[0]

heading             Uttarakhand: Man, 39, held for killing consta...
content_summary     Nainital district police on Monday arrested a...
article_link       https://timesofindia.indiatimes.com/city/dehra...
img_link           https://static.toiimg.com/thumb/imgsize-123456...
month_date                                                    Nov 08
time                                                           09:55
Year                                                            2022
Name: 0, dtype: object

In [ ]:
df = df.drop(['article_link', 'img_link'], axis=1)

In [ ]:
df.head()

,heading,content_summary,month_date,time,Year
0,"Uttarakhand: Man, 39, held for killing consta...",Nainital district police on Monday arrested a...,Nov 08,09:55,2022
1,"Nashik city: Another theft from parked car, s...",Thefts from parked cars continues in the city...,Nov 08,08:25,2022
2,Pune: Externed criminal jumps from bus fearin...,An externed criminal died in the early hours ...,Nov 08,05:17,2022
3,Gaya woman burned alive on suspicion of witch...,A middle-aged woman was badly thrashed and bu...,Nov 06,05:24,2022
4,Man held on charge of stalking in Pune,A man (33) was arrested on Tuesday from Khed ...,Nov 03,08:32,2022


In [ ]:
df['heading'][0]

' Uttarakhand: Man, 39, held for killing constable’s wife in Haldwani '

In [ ]:
for index, row in df.iterrows():
  heading=row['heading']
  article=row['content_summary']
  sentence=heading+' '+article
  year=row['Year']
  month_date=row['month_date']
  time=row['time']
  events, entities = extract_events(sentence)
  print("Index: ",index)
  print(sentence)
  print('Time of event: {}, {} at {}'.format(month_date,year, time))
  print(events)
  print(entities)
  print('-'*20)
  print('-'*20)
  

Streaming output truncated to the last 5000 lines.
--------------------
--------------------
Index:  6903
 A 22-year-old woman sustained burns on her face and hands after a man threw acid on her for not accepting his love proposal in Kalahandi district early on Monday morning. 
Time of event: Jan 16, 2018 at 14:24
[{'event': 'man throws proposal'}]
[{'entity': 'Kalahandi', 'label': 'PERSON'}, {'entity': '22-year-old', 'label': 'DATE'}, {'entity': 'Kalahandi', 'label': 'GPE'}, {'entity': 'Monday', 'label': 'DATE'}, {'entity': 'morning', 'label': 'TIME'}]
--------------------
--------------------
Index:  6904
 In a relief to police, there was a marginal decline in sex trade in the twin cities in 2017 compared to 2016.Altogether 17 sex rackets were busted last year whereas at least 29 such cases were registered in 2016. 
Time of event: Jan 16, 2018 at 14:18
[{'event': 'trade was '}]
[{'entity': '2017', 'label': 'DATE'}, {'entity': '2016.Altogether 17', 'label': 'CARDINAL'}, {'entity': 'la

In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel

In [ ]:

# Load the pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Load the crime news dataset
crime_news = pd.read_csv('/content/crime_articles.csv',nrows=50)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Define a function to extract events using BERT
def extract_events(text):
    # Tokenize the input text
    tokenized_text = tokenizer.encode(text, add_special_tokens=True)
    # Convert the tokenized input to a PyTorch tensor
    input_ids = torch.tensor([tokenized_text])
    # Extract the hidden states of the last layer of BERT
    with torch.no_grad():
        last_hidden_states = model(input_ids)[0]
    # Average the hidden states to get a fixed-length sentence embedding
    sentence_embedding = torch.mean(last_hidden_states, dim=1).squeeze()
    # Return the sentence embedding as the event representation
    return sentence_embedding.tolist()

# Apply the extract_events function to the crime news dataset
crime_news['event'] = crime_news['content_summary'].apply(extract_events)

# Print the extracted events
print(crime_news['event'])


0     [0.019860869273543358, -0.7854326963424683, -0...
1     [0.01674465276300907, -0.2876254916191101, 0.0...
2     [-0.034894492477178574, -0.5137094855308533, 0...
3     [0.22784537076950073, -0.31838735938072205, -0...
4     [-0.1338631510734558, -0.4366264045238495, 0.3...
5     [-0.001986159011721611, -0.35257458686828613, ...
6     [0.08961635828018188, -0.37324121594429016, 0....
7     [0.028927570208907127, -0.48208874464035034, -...
8     [-0.18679279088974, -0.39696353673934937, 0.13...
9     [-0.12448990345001221, -0.3868142068386078, 0....
10    [0.15127669274806976, -0.33604809641838074, 0....
11    [0.21005621552467346, -0.2599629759788513, 0.1...
12    [-0.03648967668414116, -0.3407920002937317, 0....
13    [0.17809022963047028, -0.1572565734386444, 0.2...
14    [0.15446627140045166, -0.3543147146701813, -0....
15    [0.18962444365024567, -0.2932037115097046, -0....
16    [0.23978224396705627, -0.23101209104061127, 0....
17    [0.020621759817004204, -0.3087302744388580